In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score

data_dir = 'data'
ckpt_dir = 'checkpoint'
word_embd_dir = 'checkpoint/word_embd'
model_dir = 'checkpoint/model3v1'

word_embd_dim = 100
pos_embd_dim = 25
dep_embd_dim = 25
word_vocab_size = 400001
pos_vocab_size = 10
dep_vocab_size = 21
relation_classes = 19
state_size = 100
batch_size = 10
channels = 3
lambda_l2 = 0.0001
max_len_path = 70
starter_learning_rate = 0.001
decay_steps = 2000
decay_rate = 0.96

In [2]:
with tf.name_scope("input"):
    fp_length = tf.placeholder(tf.int32, shape=[batch_size], name="fp_ength")
    fp = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="full_path")
    sp_length = tf.placeholder(tf.int32, shape=[batch_size, 2], name="sp_length")
    sp = tf.placeholder(tf.int32, [batch_size, 2, None], name="shortest_path")
    sp_pos = tf.placeholder(tf.int32, [batch_size, 2, None], name="sp_pos")
    sp_childs = tf.placeholder(tf.int32, [batch_size, 2, None, None], name="sp_childs")
    relation = tf.placeholder(tf.int32, [batch_size], name="relation")
    y_entity = tf.placeholder(tf.int32, [batch_size, None], name="y_enity")

with tf.name_scope("word_embedding"):
    W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
    embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
    embedding_init = W.assign(embedding_placeholder)
    embd_fp_word = tf.nn.embedding_lookup(W,fp[0])
    word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

with tf.name_scope("pos_embedding"):
    W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
    embd_fp_pos = tf.nn.embedding_lookup(W, fp[1])
    pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

with tf.name_scope("dep_embedding"):
    W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
    embd_sp = tf.nn.embedding_lookup(W, sp)
    dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})
    
embd_fp = tf.concat([embd_fp_word, embd_fp_pos], axis=2)
embd_fp_rev = tf.reverse(embd_fp, [1])
fp_length_rev = tf.reverse(fp_length, [0])

In [3]:
def lstm_weights(channel, embedding_dim, state_size):
    init_const = tf.zeros([1, state_size])
    with tf.variable_scope(channel):
        W_i = tf.get_variable("W_i",shape=[embedding_dim, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        U_i = tf.get_variable("U_i",shape=[state_size, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        b_i = tf.get_variable("b_i", initializer=init_const)

        W_f = tf.get_variable("W_f",shape=[embedding_dim, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        U_f = tf.get_variable("U_f",shape=[state_size, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        b_f = tf.get_variable("b_f", initializer=init_const)

        W_o = tf.get_variable("W_o",shape=[embedding_dim, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        U_o = tf.get_variable("U_o",shape=[state_size, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        b_o = tf.get_variable("b_o", initializer=init_const)

        W_g = tf.get_variable("W_g",shape=[embedding_dim, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        U_g = tf.get_variable("U_g",shape=[state_size, state_size] ,initializer=tf.contrib.layers.xavier_initializer())
        b_g = tf.get_variable("b_g", initializer=init_const)

In [4]:
lstm_weights("lstm_fw", word_embd_dim + pos_embd_dim, state_size)
lstm_weights("lstm_bw", word_embd_dim + pos_embd_dim, state_size)

In [5]:
def cond(i, steps, *agrs):
    return i< steps

In [6]:
init_hs = tf.zeros([1, state_size])
init_cs = tf.zeros([1, state_size])

init_hs_ = tf.zeros([1, state_size])
init_cs_ = tf.zeros([1, state_size])

In [7]:
hidden_states_fw = []
cell_states_fw = []

hidden_states_rev = []
cell_states_rev = []

In [8]:
i = tf.constant(0)

In [9]:
for batch in range(batch_size):
    def body(j, const, steps, input_embd, hs, cs, hs_, cs_):
        inputs = tf.expand_dims(input_embd[j], [0])
       
        with tf.variable_scope("lstm_fw", reuse=True):
            W_i = tf.get_variable("W_i")
            U_i = tf.get_variable("U_i")
            b_i = tf.get_variable("b_i")

            W_f = tf.get_variable("W_f")
            U_f = tf.get_variable("U_f")
            b_f = tf.get_variable("b_f")

            W_o = tf.get_variable("W_o")
            U_o = tf.get_variable("U_o")
            b_o = tf.get_variable("b_o")

            W_g = tf.get_variable("W_g")
            U_g = tf.get_variable("U_g")
            b_g = tf.get_variable("b_g")
            
            input_gate = tf.sigmoid(tf.matmul(inputs, W_i) + tf.matmul(hs, U_i) + b_i)
            forget_gate = tf.sigmoid(tf.matmul(inputs, W_f) + tf.matmul(hs, U_f) + b_f)
            output_gate = tf.sigmoid(tf.matmul(inputs, W_o) + tf.matmul(hs, U_o) + b_o)
            gt = tf.tanh(tf.matmul(inputs, W_g) + tf.matmul(hs, U_g) + b_g)
            cs = input_gate * gt + forget_gate * cs
            hs = output_gate * tf.tanh(cs)
            hs_ = tf.cond(tf.equal(j, const), lambda: hs, lambda: tf.concat([hs_, hs], 0))


            return j+1, const, steps, input_embd, hs, cs, hs_, cs_

    input_embd = embd_fp[batch]   
    _, _, _, _, _, _, hs_sq, cs_sq = tf.while_loop(cond, body, 
            [0, 0, fp_length[batch], input_embd, init_hs, init_cs, init_hs_, init_cs_],
            shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), input_embd.get_shape(),
            init_hs.get_shape(), init_cs.get_shape(), 
            tf.TensorShape([None, state_size]), tf.TensorShape([None, state_size])])
    
    hidden_states_fw.append(hs_sq)
    cell_states_fw.append(cs_sq)

In [10]:
for batch in range(batch_size):
    def body(j, const, steps, input_embd, hs, cs, hs_, cs_):
        x = tf.expand_dims(input_embd[j], [0])
       
        with tf.variable_scope("lstm_bw", reuse=True):
            W_i = tf.get_variable("W_i")
            U_i = tf.get_variable("U_i")
            b_i = tf.get_variable("b_i")

            W_f = tf.get_variable("W_f")
            U_f = tf.get_variable("U_f")
            b_f = tf.get_variable("b_f")

            W_o = tf.get_variable("W_o")
            U_o = tf.get_variable("U_o")
            b_o = tf.get_variable("b_o")

            W_g = tf.get_variable("W_g")
            U_g = tf.get_variable("U_g")
            b_g = tf.get_variable("b_g")
            
            input_gate = tf.sigmoid(tf.matmul(x, W_i) + tf.matmul(hs, U_i) + b_i)
            forget_gate = tf.sigmoid(tf.matmul(x, W_f) + tf.matmul(hs, U_f) + b_f)
            output_gate = tf.sigmoid(tf.matmul(x, W_o) + tf.matmul(hs, U_o) + b_o)
            gt = tf.tanh(tf.matmul(x, W_g) + tf.matmul(hs, U_g) + b_g)
            cs = input_gate * gt + forget_gate * cs
            hs = output_gate * tf.tanh(cs)
            hs_ = tf.cond(tf.equal(j, const), lambda: hs, lambda: tf.concat([hs_, hs], 0))


            return j+1, const, steps, input_embd, hs, cs, hs_, cs_

    input_embd = embd_fp_rev[batch]   
    _, _, _, _, _, _, hs_sq, cs_sq = tf.while_loop(cond, body, 
            [0, 0, fp_length_rev[batch], input_embd, init_hs, init_cs, init_hs_, init_cs_],
            shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), input_embd.get_shape(),
            init_hs.get_shape(), init_cs.get_shape(), 
            tf.TensorShape([None, state_size]), tf.TensorShape([None, state_size])])
    
    hidden_states_rev.append(hs_sq)
    cell_states_rev.append(cs_sq)

In [11]:
hidden_states_bw = tf.reverse(hidden_states_rev, [1])
cell_states_bw = tf.reverse(cell_states_rev, [1])

In [12]:
hidden_states_fw

[<tf.Tensor 'while/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_1/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_2/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_3/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_4/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_5/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_6/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_7/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_8/Exit_6:0' shape=(?, 100) dtype=float32>,
 <tf.Tensor 'while_9/Exit_6:0' shape=(?, 100) dtype=float32>]

In [13]:
hidden_states = tf.concat([hidden_states_fw, hidden_states_bw], 2)

In [14]:
hidden_states[i]

<tf.Tensor 'strided_slice_40:0' shape=(?, 200) dtype=float32>

In [15]:
tree_input_size = state_size * 2 + dep_embd_dim

In [16]:
def lstm_tree(channel, tree_input_size, state_size):
    init_const = tf.zeros([1, state_size])

    with tf.variable_scope(channel):
        W_i = tf.get_variable("W_i", shape=[tree_input_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_i = tf.get_variable("U_i", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        b_i = tf.get_variable("b_i", initializer=init_const)
        U_it = tf.get_variable("U_it", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())

        W_f = tf.get_variable("W_f", shape=[tree_input_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_f = tf.get_variable("U_f", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        b_f = tf.get_variable("b_f", initializer=init_const)
        U_ft = tf.get_variable("U_ft", shape=[max_len_path, state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())

        W_o = tf.get_variable("W_o", shape=[tree_input_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_o = tf.get_variable("U_o", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        b_o = tf.get_variable("b_o", initializer=init_const)
        U_ot = tf.get_variable("U_ot", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())

        W_u = tf.get_variable("W_u", shape=[tree_input_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        U_u = tf.get_variable("U_u", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())
        b_u = tf.get_variable("b_u", initializer=init_const)
        U_ut = tf.get_variable("U_ut", shape=[state_size, state_size], initializer=tf.contrib.layers.xavier_initializer())

In [17]:
lstm_tree("lstm_tree_btup", tree_input_size, state_size)
lstm_tree("lstm_tree_tpdn", tree_input_size, state_size)

In [18]:
hds_btup_series = []
cds_btup_series = []
hds_btup_lca_series = []
cds_btup_lca_series = []

hds_tpdn_series = []
cds_tpdn_series = []
hds_tpdn_dp_series = []
cds_tpdn_dp_series = []

In [19]:
init_hds_ = tf.zeros([1, 1, state_size])
init_cds_ = tf.zeros([1, 1, state_size])

init_hds = tf.expand_dims(tf.zeros([1, state_size]), 0)
init_cds = tf.expand_dims(tf.zeros([1, state_size]), 0)

In [20]:
for batch in range(batch_size):    

    def loop_over_seq(b, p, const, index, steps, input_pos, input_embd, input_childs, state, hds, cds, hds_, cds_):
        inputs = tf.expand_dims(tf.concat([tf.gather(hidden_states[b], input_pos[p][index]), input_embd[p][index]],0),0)
        childs = input_childs[p][index]
        num_child = tf.shape(childs)[0]

        ht = state[0]
        ct = state[1]
        num_child_sp = tf.shape(hds)[0]
        with tf.variable_scope("lstm_tree_btup", reuse=True):
            W_i = tf.get_variable("W_i")
            U_i = tf.get_variable("U_i")
            b_i = tf.get_variable("b_i")
            U_it = tf.get_variable("U_it")

            W_f = tf.get_variable("W_f")
            U_f = tf.get_variable("U_f")
            b_f = tf.get_variable("b_f")
            U_ft = tf.get_variable("U_ft")

            W_o = tf.get_variable("W_o")
            U_o= tf.get_variable("U_o")
            b_o = tf.get_variable("b_o")
            U_ot = tf.get_variable("U_ot")

            W_u = tf.get_variable("W_u")
            U_u = tf.get_variable("U_u")
            b_u = tf.get_variable("b_u")
            U_ut = tf.get_variable("U_ut")    

            it = tf.matmul(inputs, W_i) + b_i + tf.matmul(hds[0], U_i)
            ft = tf.matmul(inputs, W_f) + b_f + tf.matmul(hds[0], U_f)
            ot = tf.matmul(inputs, W_o) + b_o + tf.matmul(hds[0], U_o)
            ut = tf.matmul(inputs, W_u) + b_u + tf.matmul(hds[0], U_u)

            def matmul(k, steps, it, ft, ot, ut):
                it += tf.matmul(hds[k], U_i)
                ft += tf.matmul(hds[k], U_f) 
                ot += tf.matmul(hds[k], U_o) 
                ut += tf.matmul(hds[k], U_u) 
                return k+1, steps, it, ft, ot, ut
            _, _, it, ft, ot, ut = tf.while_loop(cond, matmul, [1, num_child_sp, it, ft, ot, ut])


            def child_sum(k, steps, out, U):
                out += tf.matmul(tf.gather(ht, childs[k]), U)
                return k+1, steps, out, U

            _, _, ht_i, _ = tf.while_loop(cond, child_sum, [0, num_child, it, U_it])
            _, _, ht_o, _ = tf.while_loop(cond, child_sum, [0, num_child, ot, U_ot])
            _, _, ht_u, _ = tf.while_loop(cond, child_sum, [0, num_child, ut, U_ut])

            input_gate = tf.sigmoid(ht_i)
            output_gate = tf.sigmoid(ht_o)
            u_input = tf.tanh(ht_u)

            cell_state = input_gate * u_input 

            def _cell_state_(k, steps, cell_state):
                _, _, f, _ = tf.while_loop(cond, child_sum, [i, num_child, ft, U_ft[k]])
                cell_state += tf.sigmoid(f) * cds[k]
                return k+1, steps, cell_state
            _, _, cell_state = tf.while_loop(cond, _cell_state_, [i, num_child_sp, cell_state])

            def _cds_(k, steps, ctl):
                _, _, fj, _ = tf.while_loop(cond, child_sum, [i, num_child, ft, U_ft[k+2]])
                ctl += tf.sigmoid(fj) * tf.gather(ct, childs[k])
                return k+1, steps, ctl

            _, _, cds = tf.while_loop(cond, _cds_, [i, num_child, cell_state])

            hds = tf.expand_dims(output_gate * tf.tanh(cds), 0)

            cds = tf.expand_dims(cds, 0)

            hds_ = tf.cond(tf.equal(index, const), lambda: hds, lambda: tf.concat([hds_, hds], 0))
            cds_ = tf.cond(tf.equal(index, const), lambda: cds, lambda: tf.concat([cds_, cds], 0))

        return b, p, const, index+1, steps, input_pos, input_embd, input_childs, state, hds, cds, hds_, cds_


    input_pos = sp_pos[batch]
    input_embd = embd_sp[batch]
    input_childs = sp_childs[batch]
    ht = tf.expand_dims(hidden_states_fw[batch], 1)
    ct = tf.expand_dims(cell_states_fw[batch], 1)
    state = tf.stack([ht, ct], axis=0)  
    
    _, _, _, _, _, _, _, _, _, _, _, hds_btup, cds_btup = tf.while_loop(cond, loop_over_seq, 
        [batch, 0, 0, 0, sp_length[batch][0] - 1, input_pos, input_embd, input_childs,
        state, init_hds, init_cds, init_hds_, init_cds_], 
        shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(),
        input_pos.get_shape(), input_embd.get_shape(), input_childs.get_shape(), 
        state.get_shape(), init_hds.get_shape(), init_cds.get_shape(), 
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size])])

    hds_lca = hds_btup[sp_length[batch][0] - 2]
    cds_lca = cds_btup[sp_length[batch][0] - 2]

    _, _, _, _, _,  _, _, _, _, _, _, hds_btup, cds_btup = tf.while_loop(cond, loop_over_seq, 
        [batch, 1, 0, 0, sp_length[batch][1], input_pos, input_embd, input_childs, state,
        init_hds, init_cds, hds_btup, cds_btup], 
        shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(),
        input_pos.get_shape(), input_embd.get_shape(), 
        input_childs.get_shape(), state.get_shape(), init_hds.get_shape(), init_cds.get_shape(), 
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size])])

    hds_lca = tf.stack([hds_lca, hds_btup[sp_length[batch][0] + sp_length[batch][1] - 2]], axis=0)
    cds_lca = tf.stack([cds_lca, cds_btup[sp_length[batch][0] + sp_length[batch][1] - 2]], axis=0)

    _, _, _, _, _, _, _, _, _, _, _, hds_btup_lca, cds_btup_lca = tf.while_loop(cond, loop_over_seq, 
        [batch, 0,  sp_length[batch][0]-1, sp_length[batch][0]-1, sp_length[batch][0], input_pos, input_embd,
        input_childs, state, hds_lca, cds_lca, init_hds_, init_cds_], 
        shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(), 
        i.get_shape(), input_pos.get_shape(), input_embd.get_shape(),
        input_childs.get_shape(), state.get_shape(), 
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size]),
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size])])
    
    hds_btup_series.append(hds_btup)
    cds_btup_series.append(cds_btup)
    hds_btup_lca_series.append(hds_btup_lca)
    cds_btup_lca_series.append(cds_btup_lca)
        

In [21]:
for batch in range(batch_size):
    
    def loop_over_seq(b, p, const, index, steps, input_pos, input_embd, input_childs, state, hds, cds, hds_, cds_):
        inputs = tf.expand_dims(tf.concat([tf.gather(hidden_states[b], input_pos[p][index]), input_embd[p][index]],0), 0)
        childs = input_childs[p][index]
        num_child = tf.shape(childs)[0]

        ht = state[0]
        ct = state[1]
        num_child_sp = tf.shape(hds)[0]
        with tf.variable_scope("lstm_tree_tpdn", reuse=True):
            W_i = tf.get_variable("W_i")
            U_i = tf.get_variable("U_i")
            b_i = tf.get_variable("b_i")
            U_it = tf.get_variable("U_it")

            W_f = tf.get_variable("W_f")
            U_f = tf.get_variable("U_f")
            b_f = tf.get_variable("b_f")
            U_ft = tf.get_variable("U_ft")

            W_o = tf.get_variable("W_o")
            U_o= tf.get_variable("U_o")
            b_o = tf.get_variable("b_o")
            U_ot = tf.get_variable("U_ot")

            W_u = tf.get_variable("W_u")
            U_u = tf.get_variable("U_u")
            b_u = tf.get_variable("b_u")
            U_ut = tf.get_variable("U_ut")    

            it = tf.matmul(inputs, W_i) + b_i + tf.matmul(hds[0], U_i)
            ft = tf.matmul(inputs, W_f) + b_f + tf.matmul(hds[0], U_f)
            ot = tf.matmul(inputs, W_o) + b_o + tf.matmul(hds[0], U_o)
            ut = tf.matmul(inputs, W_u) + b_u + tf.matmul(hds[0], U_u)

            def matmul(k, steps, it, ft, ot, ut):
                it += tf.matmul(hds[k], U_i)
                ft += tf.matmul(hds[k], U_f) 
                ot += tf.matmul(hds[k], U_o) 
                ut += tf.matmul(hds[k], U_u) 
                return k+1, steps, it, ft, ot, ut
            _, _, it, ft, ot, ut = tf.while_loop(cond, matmul, [1, num_child_sp, it, ft, ot, ut])


            def child_sum(k, steps, out, U):
                out += tf.matmul(tf.gather(ht, childs[k]), U)
                return k+1, steps, out, U

            _, _, ht_i, _ = tf.while_loop(cond, child_sum, [0, num_child, it, U_it])
            _, _, ht_o, _ = tf.while_loop(cond, child_sum, [0, num_child, ot, U_ot])
            _, _, ht_u, _ = tf.while_loop(cond, child_sum, [0, num_child, ut, U_ut])

            input_gate = tf.sigmoid(ht_i)
            output_gate = tf.sigmoid(ht_o)
            u_input = tf.tanh(ht_u)

            cell_state = input_gate * u_input 

            def _cell_state_(k, steps, cell_state):
                _, _, f, _ = tf.while_loop(cond, child_sum, [i, num_child, ft, U_ft[k]])
                cell_state += tf.sigmoid(f) * cds[k]
                return k+1, steps, cell_state
            _, _, cell_state = tf.while_loop(cond, _cell_state_, [i, num_child_sp, cell_state])

            def _cds_(k, steps, ctl):
                _, _, fj, _ = tf.while_loop(cond, child_sum, [i, num_child, ft, U_ft[k+2]])
                ctl += tf.sigmoid(fj) * tf.gather(ct, childs[k])
                return k+1, steps, ctl

            _, _, cds = tf.while_loop(cond, _cds_, [i, num_child, cell_state])

            hds = tf.expand_dims(output_gate * tf.tanh(cds), 0)

            cds = tf.expand_dims(cds, 0)

            hds_ = tf.cond(tf.equal(index, const), lambda: hds, lambda: tf.concat([hds_, hds], 0))
            cds_ = tf.cond(tf.equal(index, const), lambda: cds, lambda: tf.concat([cds_, cds], 0))

        return b, p, const, index+1, steps, input_pos, input_embd, input_childs, state, hds, cds, hds_, cds_

    
    input_pos = tf.reverse(sp_pos[batch], [1])
    input_embd = tf.reverse(embd_sp[batch], [1])
    input_childs = tf.reverse(sp_childs[batch], [1])
    ht = tf.expand_dims(hidden_states_bw[batch], 1)
    ct = tf.expand_dims(cell_states_bw[batch], 1)
    state = tf.stack([ht, ct], axis=0)  
    
    _, _, _, _, _, _, _, _, _, _, _, hds_tpdn_lca, cds_tpdn_lca = tf.while_loop(cond, loop_over_seq, 
        [batch, 0, 0, 0, 1, input_pos, input_embd, input_childs, state, 
        init_hds, init_cds, init_hds_, init_cds_], 
        shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(), 
        input_pos.get_shape(), input_embd.get_shape(), input_childs.get_shape(), state.get_shape(), 
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size]),
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size])])

    _, _, _, _, _, _, _, _, _, _, _, hds_tpdn, cds_tpdn = tf.while_loop(cond, loop_over_seq, 
        [batch, 0, 1, 1, sp_length[batch][0], input_pos, input_embd, input_childs, state, 
        hds_tpdn_lca, cds_tpdn_lca, init_hds_, init_cds_], 
        shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(), 
        input_pos.get_shape(), input_embd.get_shape(), input_childs.get_shape(), state.get_shape(), 
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size]),
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size])])
        
    hds_tpdn_dp = hds_tpdn[sp_length[batch][0] - 2]
    cds_tpdn_dp = cds_tpdn[sp_length[batch][0] - 2]
        
    _, _, _, _, _, _, _, _, _, _, _, hds_tpdn, cds_tpdn = tf.while_loop(cond, loop_over_seq, 
        [batch, 1, 0, 0, sp_length[batch][1], input_pos, input_embd, input_childs, state, 
        hds_tpdn_lca, cds_tpdn_lca, init_hds_, init_cds_], 
        shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), i.get_shape(),i.get_shape(), 
        input_pos.get_shape(), input_embd.get_shape(), input_childs.get_shape(), state.get_shape(), 
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size]),
        tf.TensorShape([None, 1, state_size]), tf.TensorShape([None, 1, state_size])])
    
    hds_tpdn_dp = tf.stack([hds_tpdn_dp, hds_tpdn[sp_length[batch][0] + sp_length[batch][1] - 2]], axis=0)
    cds_tpdn_dp = tf.stack([cds_tpdn_dp, cds_tpdn[sp_length[batch][0] + sp_length[batch][1] - 2]], axis=0)
    
    hds_tpdn_series.append(hds_tpdn)
    cds_tpdn_series.append(cds_tpdn)
    hds_tpdn_dp_series.append(hds_tpdn_dp)
    cds_tpdn_dp_series.append(cds_tpdn_dp)
    

In [40]:
dp = []
for batch in range(batch_size):
    temp  = tf.concat([hds_btup_lca_series[batch][0], hds_tpdn_dp_series[batch][0], hds_tpdn_dp_series[batch][1]],1)
    if(batch==0):
        dp = temp
    else:
        dp = tf.concat([dp, temp], 0)

In [41]:
dp

<tf.Tensor 'concat_33:0' shape=(10, 300) dtype=float32>

In [22]:
with tf.name_scope("hidden_layer_seq"):
    W = tf.Variable(tf.truncated_normal([200, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_hidden_layer = []
    y_hl = tf.zeros([1, 100])
    for batch in range(batch_size):
        def hidden_layer_loop(j, const, steps, ss, y_hl):
            temp = tf.tanh(tf.matmul(ss[j], W) + b)
            y_hl = tf.cond(tf.equal(j, const), lambda: temp, lambda: tf.concat([y_hl, temp], 0))
            return j+1, const, steps, ss, y_hl

        ss = tf.expand_dims(hidden_states[batch], 1)
        _, _, _, _, y_hls = tf.while_loop(cond, hidden_layer_loop, 
                                [0, 0, fp_length[batch], ss, y_hl],
                                shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), 
                                ss.get_shape(), tf.TensorShape([None, 100])])
        y_hidden_layer.append(y_hls)

In [46]:
with tf.name_scope("dropout_hidden_layer_seq"):
    y_hidden_layer_drop = tf.nn.dropout(y_hidden_layer, 0.3)

In [48]:
with tf.name_scope("softmax_layer_seq"):
    W = tf.Variable(tf.truncated_normal([100, 2], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([2]), name="b")
    
    logits_entity = []
    predictions_entity = []
    lg = tf.zeros([1, 2])
    
    for batch in range(batch_size):
        def softmax_layer_loop(j, const, steps, y_hl, lg):
            temp = tf.matmul(y_hl[j], W) + b
            lg = tf.cond(tf.equal(j, const), lambda: temp, lambda: tf.concat([lg, temp], 0))
            return j+1, const, steps, y_hl, lg

        y_hl = tf.expand_dims(y_hidden_layer_drop[batch], 1)
        _, _, _, _, logit = tf.while_loop(cond, softmax_layer_loop, 
                                [0, 0, fp_length[batch], y_hl,lg],
                                shape_invariants=[i.get_shape(), i.get_shape(), i.get_shape(), 
                                y_hl.get_shape(), tf.TensorShape([None, 2])])
        logits_entity.append(logit)
        predictions_entity.append(tf.arg_max(logit, 1))

In [50]:
with tf.name_scope("loss_seq"):
    loss_seq = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_entity, labels=y_entity))

In [51]:
with tf.name_scope("hidden_layer_dep"):
    W = tf.Variable(tf.truncated_normal([300, 100], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([100]), name="b")
    y_p = tf.tanh(tf.matmul(dp, W) + b)

In [53]:
with tf.name_scope("softmax_layer_dep"):
    W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
    b = tf.Variable(tf.zeros([relation_classes]), name="b")
    logits = tf.matmul(y_p, W) + b    

In [54]:
with tf.name_scope("loss_dep"):
    loss_dep = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=relation))

In [64]:
tv_all = tf.trainable_variables()

tv_regu = []

non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0']

for t in tv_all:
    if t.name not in non_reg:
        if(t.name.find('b_')==-1):
            tv_regu.append(t)

In [66]:
with tf.name_scope("total_loss"):
    l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
    total_loss = l2_loss + loss_seq + loss_dep

In [6]:
global_step = tf.Variable(0, trainable=False, name="global_step")

learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(total_loss, global_step=global_step)

In [7]:
f = open('data/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

pos_tags_vocab = []
for line in open('data/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

dep_vocab = []
for line in open('data/dependency_types.txt'):
    dep_vocab.append(line.strip())

relation_vocab = []
for line in open('data/relation_types.txt'):
    relation_vocab.append(line.strip())

rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [9]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()
# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [10]:
# model = tf.train.latest_checkpoint(model_dir)
# saver.restore(sess, model)

In [11]:
latest_embd = tf.train.latest_checkpoint(word_embd_dir)
word_embedding_saver.restore(sess, latest_embd)

INFO:tensorflow:Restoring parameters from checkpoint/word_embd/word_embd


In [12]:
f = open('data/train_lca_paths', 'rb')
word_p, dep_p, pos_p = pickle.load(f)
f.close()
relations = []
for line in open('data/train_relations.txt'):
    relations.append(line.strip().split()[1])

length = len(word_p)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p[i]):
        word = word.lower()
        word_p[i][j] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p[i]):
        dep_p[i][l] = d if d in dep2id else 'OTH'
        
word_p_ids = np.ones([length, max_len_path],dtype=int)
pos_p_ids = np.ones([length, max_len_path],dtype=int)
dep_p_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path_len = np.array([len(w) for w in word_p], dtype=int)

for i in range(length):
    for j, w in enumerate(word_p[i]):
        word_p_ids[i][j] = word2id[w]
        
    for j, w in enumerate(pos_p[i]):
        pos_p_ids[i][j] = pos_tag(w)
        
    for j, w in enumerate(dep_p[i]):
        dep_p_ids[i][j] = dep2id[w]

In [13]:
num_epochs = 10
for i in range(num_epochs):
    loss_per_epoch = 0
    for j in range(num_batches):
        feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
        _, _loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        loss_per_epoch +=_loss
        if (j+1)%num_batches==0:
            print("Epoch:", i+1,"Step:", step, "loss:",loss_per_epoch/num_batches)
    saver.save(sess, model_dir + '/model')
    print("Saved Model")

Epoch: 1 Step: 800 loss: 2.85300489247
Saved Model
Epoch: 2 Step: 1600 loss: 2.73827668965
Saved Model
Epoch: 3 Step: 2400 loss: 2.70001435518
Saved Model
Epoch: 4 Step: 3200 loss: 2.68624746531
Saved Model
Epoch: 5 Step: 4000 loss: 2.68042603165
Saved Model
Epoch: 6 Step: 4800 loss: 2.67750604913
Saved Model
Epoch: 7 Step: 5600 loss: 2.67583220631
Saved Model
Epoch: 8 Step: 6400 loss: 2.67482194766
Saved Model
Epoch: 9 Step: 7200 loss: 2.67411908716
Saved Model
Epoch: 10 Step: 8000 loss: 2.67369878128
Saved Model


In [ ]:
# training accuracy
all_predictions = []
for j in range(num_batches):
     feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("training accuracy", accuracy)

In [25]:
f = open('data/test_lca_paths', 'rb')
word_p, dep_p, pos_p = pickle.load(f)
f.close()

relations = []
for line in open('data/test_relations.txt'):
    relations.append(line.strip().split()[0])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p[i]):
        word = word.lower()
        word_p[i][j] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p[i]):
        dep_p[i][l] = d if d in dep2id else 'OTH'
        
word_p_ids = np.ones([length, max_len_path],dtype=int)
pos_p_ids = np.ones([length, max_len_path],dtype=int)
dep_p_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path_len = np.array([len(w) for w in word_p], dtype=int)

for i in range(length):
    for j, w in enumerate(word_p[i]):
        word_p_ids[i][j] = word2id[w]
        
    for j, w in enumerate(pos_p[i]):
        pos_p_ids[i][j] = pos_tag(w)
        
    for j, w in enumerate(dep_p[i]):
        dep_p_ids[i][j] = dep2id[w]

# test predictions
all_predictions = []
for j in range(num_batches):
     feed_dict = {
            path_length:path_len[j*batch_size:(j+1)*batch_size],
            word_ids:word_p_ids[j*batch_size:(j+1)*batch_size],
            pos_ids:pos_p_ids[j*batch_size:(j+1)*batch_size],
            dep_ids:dep_p_ids[j*batch_size:(j+1)*batch_size],
            y:rel_ids[j*batch_size:(j+1)*batch_size]}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("test accuracy", accuracy)

ValueError: Cannot feed value of shape (2, 10) for Tensor 'input_1/path1_length:0', which has shape '(10,)'